# Inference Pipeline

### Imports

In [1]:
import numpy as np
import h5py
import sys
sys.path.append('../../d-script/')

# Neural network stuff
from fielutil import load_verbatimnet
from featextractor import extract_imfeats

Using Theano backend.


Using gpu device 0: GeForce GTX 980


### Parameters

In [2]:
# Do you want to load the features in? Or save them to a file?
load_features = True
# All the images that you require extraction should be in this HDF5 file
hdf5images='icdar13data/benchmarking-processed/icdar_be.hdf5'
# This is the file that you will load the features from or save the features to
featurefile = 'icdar13data/benchmarking-processed/icdar13be_fiel1k.npy'
# This is the neural networks and parameters you are deciding to use
paramsfile = '/fileserver/iam/iam-processed/models/fiel_1k.hdf5'

### Full image HDF5 file

Each entry in the HDF5 file is a full image/form/document

In [3]:
labels = h5py.File(hdf5images).keys()

### Load feature extractor neural network

In [4]:
vnet = load_verbatimnet( 'fc7', params='/fileserver/iam/iam-processed/models/fiel_1k.hdf5' )

Establishing Fiel's verbatim network
Compiled neural network up to FC7 layer


### Image features

Currently taken as averages of all shard features in the image. You can either load them or extract everything manually, depending on if you have the .npy array.

In [ ]:
if load_features:
    imfeats = np.load(featurefile)
else:
    imfeats = extract_imfeats( hdf5name, vnet )

### Build classifier

In [ ]:
imfeats = ( imfeats.T / np.linalg.norm( imfeats, axis=1 ) ).T
F = imfeats.dot(imfeats.T)
np.fill_diagonal( F , -1 )

### Evaluate classifier on HDF5 file

In [ ]:
numcorrect = 0
totalnum = 0

# Pick top k
k = 1
# Run through the adjacency matrix
for j, i in enumerate(F):
    topk = i.argsort()[-k:]
    if j/4 in topk/4:
        numcorrect += 1
    totalnum +=1

print "Top %d = %f" %( k, (numcorrect+0.0) / totalnum )

In [ ]:
totalnum/4

In [ ]:
whos

In [ ]:
np.save( 'icdar13be_fiel1k.npy', imfeats)